# Analisi dei Piloti del Mondiale di Formula 1



## Dataset

[Link del dataset](https://proai-datasets.s3.eu-west-3.amazonaws.com/formula1_data.csv)

Il dataset formula1_data.csv contiene le seguenti colonne: </br>
1. Driver: Nome del pilota.
2. Team: Nome del costruttore per il quale il pilota gareggia.
3. Race: Città in cui si è svolto il Gran Premio.
4. Country: Paese in cui si è svolto il Gran Premio.
5. Position: Numero compreso tra 0 e 8 che indica l'ordine di arrivo del pilota </br> (0 significa che il pilota non è arrivato tra i primi 8 e non ha ottenuto punti).


## Sistema di punteggio

Al termine di ogni Gran Premio, i punti vengono assegnati ai piloti in base al loro ordine di arrivo come segue: </br>

* 1° posto: 10 punti
* 2° posto: 8 punti
* 3° posto: 6 punti
* 4° posto: 5 punti
* 5° posto: 4 punti
* 6° posto: 3 punti
* 7° posto: 2 punti
* 8° posto: 1 punto
* 9° posto o oltre: 0 punti

## Obiettivi del progetto
### 1. Funzione per l'analisi delle performance individuali dei piloti

La prima funzione riceve in input il nome di un pilota e restituisce una lista contenente tre informazioni chiave:
* Il totale dei punti accumulati dal pilota durante il campionato.
* Il numero di vittorie, ovvero quante volte il pilota è arrivato primo in un Gran Premio.
* Il numero di podi, ovvero quante volte il pilota è arrivato tra i primi tre classificati.

Questa funzione sarà utile per analizzare le performance individuali dei piloti e avere una chiara visione delle loro posizioni nel corso della stagione.

### 2. Funzione per la creazione della classifica finale dei piloti
La seconda funzione genera un dizionario contenente i nomi dei piloti come chiavi e il loro punteggio totale come valori. Il dizionario viene poi utilizzato per creare una classifica generale dei piloti.

Infine, la classifica sarà salvata in un file di testo (Drivers_Standings_2008.txt) con il seguente formato:

Drivers Standings 2008 Formula 1 </br>
NomePilota1: PunteggioTotale </br>
NomePilota2: PunteggioTotale </br>

### 3. Funzione per la classifica dei costruttori

La terza funzione crea un dizionario con i nomi dei team/costruttori come chiavi e il loro punteggio totale come valori. Il punteggio di ciascun team è la somma dei punti ottenuti dai piloti che hanno gareggiato per quel costruttore.

Questa funzione utilizza i dati precedentemente generati per i piloti e calcola la classifica dei costruttori. Anche questa informazione è essenziale per avere una visione chiara delle prestazioni dei team durante l'anno.

---

# Realizzazione del Progetto

## Soluzione adottata

Il codice ha la seguente logica di base: inizialmente si raccolgono i dati dal dataset CSV, salvandoli all'interno di appositi dizionari; successivamente, si utilizzano tali dizionari per calcolare la performance individuale di un pilota inserito dall'utente, per generare la classifica finale dei piloti e salvarla su un file TXT, e per generare la classifica finale dei team/costruttori.

La raccolta dei dati è stata implementata all'interno della funzione ***collect_data***, che legge il dataset e restituisce i seguenti dizionari:

* Dizionario contenente i nomi dei piloti come chiavi, e il loro punteggio totale come valori

* Dizionario contenente i nomi dei piloti come chiavi, e il loro numero di vittorie come valori

* Dizionario contenente i nomi dei piloti come chiavi, e il loro numero di podi come valori

* Dizionario contenente i nomi dei team/costruttori come chiavi, e il loro punteggio totale come valori

Tutto ciò che segue la raccolta dei dati è, in sostanza, una consultazione dei suddetti dizionari: infatti, le tre funzioni richieste nel progetto si basano quasi interamente sui dizionari contenenti i dati precedentemente raccolti. <br>
All'interno della funzione ***main*** saranno poi chiamate le tre funzioni per poter essere eseguite.

Per chiarezza, si specifica che sono state effettuate le seguenti scelte:

* Nonostante il dataset sia privo di errori legati al valore della posizione di arrivo dei piloti, nella funzione ***collect_data*** viene eseguito un semplice controllo sulla variabile *position*, imponendo che essa sia un intero tra 0 e 8.

* La funzione ***drivers_rank*** non genera il dizionario {piloti : punteggio totale dei piloti}, bensì lo ottiene direttamente nell'argomento; procedendo in questo modo, è possibile risparmiare risorse computazionali ed evitare di collezionare i dati in modo ridondante.

* Analogamente, la funzione ***teams_rank*** non genera il dizionario {team : punteggio totale dei team}, ma lo ottiene nell'argomento.

Infine, si specifica che l'unico supporto utilizzato per la scrittura del codice è stata la piattaforma **ProfAI**.

## Codice

In [4]:
# Importo il dataset "formula1_data.csv" nella mia directory di lavoro
# su Colab/Jupiter basta scrivere !wget https://proai-datasets.s3.eu-west-3.amazonaws.com/formula1_data.csv

import requests

url = "https://proai-datasets.s3.eu-west-3.amazonaws.com/formula1_data.csv"
response = requests.get(url)

with open("formula1_data.csv", "wb") as file:
    file.write(response.content)

In [5]:
import csv

def collect_data(filename = "formula1_data.csv"):

    """
    Questa funzione legge il file CSV e salva i dati necessari in
    appositi dizionari utilizzando il sistema di punteggio.
    La funzione è necessaria per poter leggere il dataset una
    sola volta, con conseguente risparmio di risorse computazionali.

    Keyword Argument: filename   (str)

    Returns:
        - driver_points   (dict)
        - victories       (dict)
        - podiums         (dict)
        - team_points     (dict)
    """

    driver_points = {}  # dizionario piloti:punteggi
    victories = {}      # dizionario piloti:vittorie
    podiums = {}        # dizionario piloti:podi
    team_points = {}    # dizionario teams:punteggi

    # Tupla ordinata dei possibili punteggi
    scores_tuple = (0, 10, 8, 6, 5, 4, 3, 2, 1)

    # Apro il dataset in sola lettura all'interno di un contesto
    with open(filename, 'r', newline="", encoding='utf8') as csv_dataset:

        csv_reader = csv.DictReader(csv_dataset)

        for row in csv_reader:

            driver = row["Driver"]
            team = row["Team"]
            position = row["Position"]

            # Impongo che il parametro position sia un intero tra 0 e 8

            try:
                position = int(position)
                assert position in range(0,9), "La posizione di arrivo deve essere compresa tra 0 e 8"

            except ValueError:
                print("È stata riscontrata una posizione di tipo non intero!")
                return None, None, None, None

            except AssertionError as e:
                print(e)
                return None, None, None, None

            # Assegno il punteggio in base alla posizione di arrivo
            score = scores_tuple[position]

            # Popolo i dizionari relativi ai piloti, facendo attenzione ad inizializzare
            # a zero il valore relativo al pilota qualora la sua chiave driver fosse assente

            if driver not in driver_points:
                driver_points[driver] = 0

            if driver not in victories:
                victories[driver] = 0

            if driver not in podiums:
                podiums[driver] = 0

            driver_points[driver] += score  # Aggiorno il punteggio del pilota

            if position in range(1,4):  # Podio: prime tre posizioni

                if position == 1:   # Vittoria: prima posizione
                    victories[driver] += 1

                podiums[driver] += 1

            # Popolo il dizionario dei team, facendo attenzione ad inizializzare
            # a zero il valore relativo al team qualora la sua chiave team fosse assente

            if team not in team_points:
                team_points[team] = 0

            team_points[team] += score  # Aggiorno il punteggio del team

    return driver_points, victories, podiums, team_points


# Funzione 1


def individual_performance(driver_name, driver_points, victories, podiums):

    """
    Funzione per l'analisi delle performance individuali dei piloti:
    restituisce una lista contenente il punteggio totale, il numero
    di vittorie e il numero di podi del pilota.

    Positional Arguments:
        - driver_name      (str)
        - driver_points    (dict)
        - victories        (dict)
        - podiums          (dict)

    Return: driver_stats   (list)
    """

    # Creo la lista della performance individuale
    driver_stats = [driver_points[driver_name],
                    victories[driver_name],
                    podiums[driver_name]
                    ]

    return driver_stats


# Funzione 2


def drivers_rank(driver_points):

    """
    Funzione per la creazione della classifica finale dei piloti:
    genera la classifica e la salva all'interno di un file di testo;
    restituisce una lista ordinata per punteggio decrescente dei piloti.

    Positional Argument: driver_points   (dict)

    Return: drivers_rank_list   (list)
    """

    # Il parametro driver_points è proprio il dizionario contenente i nomi
    # dei piloti come chiavi e il loro punteggio totale come valori,
    # il quale viene generato chiamando collect_data nella Main

    # Creo una lista di tuple (Pilota, Punteggio totale), inizialmente vuota
    drivers_rank_list = []

    for driver, points in driver_points.items():
        drivers_rank_list.append((driver, points))

    # Ordino la lista in ordine decrescente in base al punteggio
    drivers_rank_list.sort(key=lambda val: val[1], reverse=True)

    # Salvo la classifica nel file di testo

    with open("Drivers_Standings_2008.txt", 'w', newline="", encoding='utf8') as txt_file:

        txt_file.write("Drivers Standings 2008 Formula 1 \n")   # Intestazione

        for driver, points in drivers_rank_list:
            txt_file.write(f"{driver}: {points}\n")

    print("\n"+ "La classifica finale dei piloti è stata stampata nel file di testo. \n")

    return drivers_rank_list


# Funzione 3


def teams_rank(team_points):

    """
    Funzione per la creazione della classifica finale dei team/costruttori:
    restituisce una lista ordinata per punteggio decrescente dei team.

    Positional Argument: team_points   (dict)

    Return: teams_rank_list   (list)
    """

    # Il parametro team_points è proprio il dizionario contenente i nomi
    # dei team come chiavi e il loro punteggio totale come valori,
    # il quale viene generato chiamando collect_data nella Main

    # Creo una lista di tuple (Team, Punteggio totale), inizialmente vuota
    teams_rank_list = []

    for team, points in team_points.items():
        teams_rank_list.append((team, points))

    # Ordino la lista in ordine decrescente in base al punteggio
    teams_rank_list.sort(key=lambda val: val[1], reverse=True)

    return teams_rank_list


In [6]:
# MAIN FUNCTION

if __name__ == "__main__":

    # Colleziono i dati all'interno dei dizionari
    driver_points, victories, podiums, team_points = collect_data()

    # Controllo che la raccolta dati sia avvenuta correttamente
    assert driver_points is not None, "Si è verificato un errore nella raccolta dei dati!"

    # Richiedo il nome del pilota all'utente, e controllo che
    # il nome sia presente tra le chiavi di driver_points

    while True:
        driver_name = input("Di quale pilota vuoi analizzare la performance? ")
        if driver_name in driver_points:
            break
        else:
            print("\n" +
                  f"Il pilota {driver_name} non partecipa al Mondiale di Formula 1! \n" +
                  "Piloti disponibili:", ", ".join(driver_points.keys()) +
                  "\n"
                  )

    # Output della performance del pilota inserito dall'utente
    driver_stats = individual_performance(driver_name, driver_points, victories, podiums)

    print("\n" +
    f"Performance individuale di {driver_name}: \n\n" +
    f"Totale dei punti: {driver_stats[0]} \n" +
    f"Numero di vittorie: {driver_stats[1]} \n" +
    f"Numero di podi: {driver_stats[2]}"
    )

    # Salvo la classifica finale dei piloti nel file di testo, e ottengo drivers_rank_list
    drivers_rank_list = drivers_rank(driver_points)

    # Mostro la classifica dei piloti all'utente
    print("Classifica finale dei piloti: \n")
    for driver, points in drivers_rank_list:
        print(f"Pilota {driver}: {points} punti")

    print("")

    # Genero la classifica finale dei team, e ottengo teams_rank_list
    teams_rank_list = teams_rank(team_points)

    # Mostro la classifica dei team all'utente
    print("Classifica finale dei team/costruttori: \n")
    for team, points in teams_rank_list:
        print(f"Team {team}: {points} punti")

# Fine



Il pilota Leo non partecipa al Mondiale di Formula 1! 
Piloti disponibili: Hamilton, Massa, Raikkonen, Kubica, Alonso, Heidfeld, Kovalainen, Vettel, Trulli, Glock


Performance individuale di Massa: 

Totale dei punti: 97 
Numero di vittorie: 6 
Numero di podi: 10

La classifica finale dei piloti è stata stampata nel file di testo. 

Classifica finale dei piloti: 

Pilota Hamilton: 98 punti
Pilota Massa: 97 punti
Pilota Raikkonen: 75 punti
Pilota Kubica: 75 punti
Pilota Alonso: 61 punti
Pilota Heidfeld: 60 punti
Pilota Kovalainen: 53 punti
Pilota Vettel: 35 punti
Pilota Trulli: 31 punti
Pilota Glock: 25 punti

Classifica finale dei team/costruttori: 

Team Ferrari: 172 punti
Team McLaren: 151 punti
Team BMW: 135 punti
Team Renault: 61 punti
Team Toyota: 56 punti
Team Toro Rosso: 35 punti
